# IO Tutorial

> Warning **This tutorial is still under construction.**

This notebook is designed to demonstrate the pynapple IO. It is build around the specifications of the [BIDS standard](https://bids-standard.github.io/bids-starter-kit/index.html) for sharing datasets. The key ideas are summarized as follow :

- [Hierarchy of folders](https://bids-standard.github.io/bids-starter-kit/folders_and_files/folders.html)

    <img src="BIDS_Folders.png" width="500" height="300" />
    
- [Filename template](https://bids-standard.github.io/bids-starter-kit/folders_and_files/files.html)

    <img src="BIDS_Files.png" width="500" height="300" />

- [Metadata files](https://bids-standard.github.io/bids-starter-kit/folders_and_files/metadata.html)
    
    <img src="BIDS_Metadata.png" width="500" height="300" />




## 1. Navigating a structured dataset

The dataset in this example can be found [here](https://www.dropbox.com/s/pr1ze1nuiwk8kw9/MyProject.zip?dl=1).

In [1]:
import numpy as np
import pynapple as nap

project_path = '../../your/path/to/MyProject'

project = nap.load_folder(project_path)

project

📂 MyProject
└── 📂 sub-A2929

The pynapple IO offers a convenient way of visualizing and navigating a folder based dataset. To visualize the whole hierarchy of Folders, you can call the view property or the expand function.

In [2]:
project.view

📂 MyProject
└── 📂 sub-A2929
    └── 📂 A2929-200711
        ├── 📂 derivatives
        │   ├── spikes.npz      |        TsGroup
        │   ├── sleep_ep.npz    |        IntervalSet
        │   ├── position.npz    |        TsdFrame
        │   └── wake_ep.npz     |        IntervalSet
        └── 📂 pynapplenwb
            └── A2929-200711.nwb        |        NWB file

Here it shows all the subjects (in this case only A2929), all the sessions and all of the derivatives folders. It shows as well all the NPZ files that contains a pynapple object and the NWB files.

The object project behaves like a nested dictionnary. It is then easy to loop and navigate through a hierarchy of folders when doing analyses. In this case, we are gonna take only the session A2929-200711.

In [3]:
session = project["sub-A2929"]["A2929-200711"]

session

📂 A2929-200711
├── 📂 derivatives
└── 📂 pynapplenwb

I can expand to see what the folders contains.

In [4]:
session.expand()

📂 A2929-200711
├── 📂 derivatives
│   ├── spikes.npz      |        TsGroup
│   ├── sleep_ep.npz    |        IntervalSet
│   ├── position.npz    |        TsdFrame
│   └── wake_ep.npz     |        IntervalSet
└── 📂 pynapplenwb
    └── A2929-200711.nwb        |        NWB file

## 2. Loading files

By default, pynapple save objects as NPZ. It is a convenient way to save all the properties of an object such as the time support. The pynapple IO offers an easy way to load any NPZ files that matches the structures defined for a pynapple object.

In [5]:
spikes = session["derivatives"]["spikes"]
position = session["derivatives"]["position"]
wake_ep = session["derivatives"]["wake_ep"]
sleep_ep = session["derivatives"]["sleep_ep"]

Objects are only loaded when they are called.

In [6]:
print(session["derivatives"]["spikes"])

  Index    rate    group  location
-------  ------  -------  ----------
      0    7.3         0  adn
      1    5.73        0  adn
      2    8.12        0  adn
      3    6.68        0  adn
      4   10.77        0  adn
      5   11           0  adn
      6   16.52        0  adn
      7    2.2         1  ca1
      8    2.02        1  ca1
      9    1.07        1  ca1
     10    3.92        1  ca1
     11    3.31        1  ca1
     12    1.09        1  ca1
     13    1.28        1  ca1
     14    1.32        1  ca1


## 3. Metadata

A good practice for sharing datasets is to write as many metainformation as possible. Following BIDS specifications, any data files should be accompagned by a JSON sidecar file. 

In [7]:
import os
for f in os.listdir(session['derivatives'].path):
    print(f)

wake_ep.json
position.json
sleep_ep.json
spikes.npz
sleep_ep.npz
spikes.json
position.npz
wake_ep.npz


To read the metainformation associated with a file, you can use the functions _doc_,_info_ or _metadata_ : 

In [8]:
session['derivatives'].doc("spikes")

╭─ ../../your/path/to/MyProject/sub-A2929/A2929-200711/derivatives/spikes.npz ─╮
│ time : 2023-07-11 12:40:10.338066                                            │
│ info : Neurons recorded simultaneously in ADN and CA1                        │
╰──────────────────────────────────────────────────────────────────────────────╯

In [9]:
session['derivatives'].doc("position")

╭─ ../../your/path/to/MyProject/sub-A2929/A2929-200711/derivatives/position.npz ─╮
│ time : 2023-07-11 12:40:10.364061                                              │
│ info : Position and head-direction of the mouse recorded with Optitrack        │
╰────────────────────────────────────────────────────────────────────────────────╯

## 4. Saving a pynapple object.

In this case, we define a new Tsd and a new IntervalSet that we would like to save in the session folder.

In [10]:
tsd = position['x'] + position['y']
epoch = nap.IntervalSet(start=np.array([0, 3]), end = np.array([1, 6]))

session.save("x_plus_y", tsd, description = "Random position")
session.save("stimulus-fish", epoch, description = "Fish pictures to V1")

We can visualize the newly saved objects.

In [11]:
session.expand()

📂 A2929-200711
├── 📂 derivatives
│   ├── spikes.npz      |        TsGroup
│   ├── sleep_ep.npz    |        IntervalSet
│   ├── position.npz    |        TsdFrame
│   └── wake_ep.npz     |        IntervalSet
├── 📂 pynapplenwb
│   └── A2929-200711.nwb        |        NWB file
├── x_plus_y.npz    |        Tsd
└── stimulus-fish.npz       |        IntervalSet

In [16]:
session.doc('stimulus-fish')

╭─ ../../your/path/to/MyProject/sub-A2929/A2929-200711/stimulus-fish.npz ─╮
│ time : 2023-07-11 13:32:01.692966                                       │
│ info : Fish pictures to V1                                              │
╰─────────────────────────────────────────────────────────────────────────╯

In [17]:
session['x_plus_y']

Time (s)
670.64070     0.382166
670.64900     0.380987
670.65735     0.379844
670.66565     0.378787
670.67400     0.377807
                ...   
1199.96160    0.049132
1199.96995    0.049651
1199.97825    0.050196
1199.98660    0.050725
1199.99495    0.051192
Length: 63527, dtype: float64